In [1]:
from keras import backend as K
from keras.layers import Input, Dense
from keras.models import Model
from keras.layers import Dense, Dropout, Reshape, Permute
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import ReLU
from keras.layers.recurrent import GRU
from keras.utils.data_utils import get_file
from keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D
import librosa
import time
import numpy as np
np.set_printoptions(threshold=np.nan)
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

Using TensorFlow backend.


In [2]:
X_data1 = np.load('mfcc_select1_pro_bkgn.npy')
X_data2 = np.load('mfcc_select2_pro_bkgn.npy')
y_data1 = np.load('label_select1_pro_bkgn.npy')
y_data2 = np.load('label_select2_pro_bkgn.npy')

X_data = np.concatenate((X_data1,X_data2),axis=0)
y_data = np.concatenate((y_data1,y_data2),axis=0)

X_data.shape, y_data.shape

((1926, 2, 44), (1926, 2))

In [7]:
X_data = np.load('mfcc_select2_pro.npy')
y_data = np.load('label_select2_pro.npy')

X_data.shape, y_data.shape

((657, 2, 44), (657, 2))

In [23]:
from sklearn.preprocessing import StandardScaler
print(X_data[0][0])
X_data = tf.keras.utils.normalize(X_data, axis=-1, order=2)
#X_data = StandardScaler().fit_transform(X_data)
print("after")
print(X_data[0][0])

[-349.38297733 -339.46299205 -335.9631354  -332.85456749 -335.33701397
 -338.99786663 -335.77088785 -333.44439033 -329.44559082 -322.16420586
 -319.92502913 -316.64271557 -311.43076845 -308.73829721 -309.3209236
 -307.67348592 -303.83712975 -305.39611913 -310.7827827  -312.72902822
 -320.60781548 -332.39978589 -345.72288923 -354.45209134 -345.82691536
 -222.58115458 -144.63731484 -104.18134994  -94.19182443 -125.13382514
 -176.27034638 -187.67775668 -142.13757249 -106.01491192 -103.24835776
 -105.74849192 -113.21542036 -124.97840797 -130.46651078 -136.70618579
 -157.23289474 -202.31283386 -221.40212359 -224.78087842]
after
[-0.19771582 -0.1921021  -0.19012153 -0.18836239 -0.18976721 -0.19183888
 -0.19001274 -0.18869617 -0.18643325 -0.18231272 -0.18104557 -0.17918811
 -0.17623866 -0.174715   -0.1750447  -0.17411242 -0.17194142 -0.17282366
 -0.17587197 -0.17697335 -0.18143196 -0.18810503 -0.19564457 -0.20058443
 -0.19570344 -0.12595867 -0.08185025 -0.05895622 -0.05330315 -0.07081323
 -0.

In [3]:
print(X_data.shape)
X_data = np.array(X_data.reshape(X_data.shape[0], 2, 44, 1))
print(X_data.shape)
y=[]

ns=s=0
for i in range(len(y_data)):
    if y_data[i][0]==1:
        y.append('0')
        ns+=1
    else:
        y.append('1')
        s+=1

print("ns = "+str(ns))
print("s = "+str(s))
y = np.array(y)

(1926, 2, 44)
(1926, 2, 44, 1)
ns = 1542
s = 384


In [44]:
from sklearn.model_selection import train_test_split
#X_sub, X_test, y_sub, y_test = train_test_split(X_data, y_data, test_size=0.2)
X_train, X_test, y_train, y_test = train_test_split(X_data, y, test_size=0.02)

In [45]:
len(X_train), len(X_test), len(y_train), len(y_test)

(1887, 39, 1887, 39)

In [46]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1887, 2, 44, 1), (39, 2, 44, 1), (1887,), (39,))

In [47]:
from keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D
from keras.models import Sequential

model = Sequential()

# Model Input
input_shape=(2, 44, 1)

# Using CNN to build model
# W=(W−F+2P)/S+1
# 16 depths (2-1)/1+1 x (44-5)/2+1 = 2 x 20 x 16
model.add(Conv2D(32, (1, 5), strides=(1, 2), input_shape=input_shape))
#model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))

# 2 x 8 x 32
model.add(Conv2D(32, (1, 5), strides=(1, 2), padding="valid"))
#model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))

model.add(Reshape((16, 32)))

# GRU block 1, 2, output
model.add(GRU(32, return_sequences=True, name='gru1'))
model.add(GRU(32, return_sequences=False, name='gru2'))
#model.add(GRU(32, return_sequences=False, name='gru3'))
model.add(Dropout(0.3))

# Output
model.add(Dense(1))
model.add(Activation('sigmoid'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 2, 20, 32)         192       
_________________________________________________________________
activation_25 (Activation)   (None, 2, 20, 32)         0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 2, 8, 32)          5152      
_________________________________________________________________
activation_26 (Activation)   (None, 2, 8, 32)          0         
_________________________________________________________________
reshape_9 (Reshape)          (None, 16, 32)            0         
_________________________________________________________________
gru1 (GRU)                   (None, 16, 32)            6240      
_________________________________________________________________
gru2 (GRU)                   (None, 32)                6240      
__________

In [48]:
model.compile(
    optimizer="Adam",
    loss="binary_crossentropy",
    metrics=['accuracy'])

model.fit(
    x=X_train,
    y=y_train,
    epochs=64,
    batch_size=64,
    validation_data= (X_test, y_test),
)

score = model.evaluate(x=X_test,y=y_test)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

#timestr = time.strftime('%Y%m%d-%H%M%S')


Train on 1887 samples, validate on 39 samples
Epoch 1/64
1887/1887 [==============================] - 3s 2ms/step - loss: 0.5302 - acc: 0.7822 - val_loss: 0.3798 - val_acc: 0.8462
Epoch 2/64
1887/1887 [==============================] - 1s 756us/step - loss: 0.4845 - acc: 0.8066 - val_loss: 0.3597 - val_acc: 0.8462
Epoch 3/64
1887/1887 [==============================] - 1s 760us/step - loss: 0.4750 - acc: 0.8066 - val_loss: 0.3432 - val_acc: 0.8462
Epoch 4/64
1887/1887 [==============================] - 1s 756us/step - loss: 0.4692 - acc: 0.8124 - val_loss: 0.3657 - val_acc: 0.8462
Epoch 5/64
1887/1887 [==============================] - 1s 760us/step - loss: 0.4576 - acc: 0.8156 - val_loss: 0.3641 - val_acc: 0.8462
Epoch 6/64
1887/1887 [==============================] - 1s 763us/step - loss: 0.4449 - acc: 0.8203 - val_loss: 0.3687 - val_acc: 0.8462
Epoch 7/64
1887/1887 [==============================] - 1s 753us/step - loss: 0.4460 - acc: 0.8198 - val_loss: 0.3481 - val_acc: 0.8462
Epoc

In [39]:
modelName = 'rcnn-2d-select1and2-pro-exns-bkgn-1.h5'.format(timestr)
model.save('models/{}'.format(modelName))
print('Model exported and finished')

In [65]:
import librosa
#import keras
import numpy as np
import tensorflow as tf
from pydub import AudioSegment
#from pydub.silence import split_on_silence
#import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
from keras.models import load_model
#import matplotlib.pyplot as plt
#%matplotlib inline
#plt.style.use('ggplot')
np.set_printoptions(threshold=np.nan)

sound_file = AudioSegment.from_wav("/home/mansi/anaconda3/beproject/stutter_det/read1s2dmf/M_0048_11y1m_2/M_0048_11y1m_2.wav")

audio_chunks = sound_file[::1000]

mfcc_array = []
print("ff")
print(mfcc_array)
for i, chunk in enumerate(audio_chunks):
    chunkfile = "chunks_test/chunk{0}.wav".format(i)
    chunk.export(chunkfile, format="wav")
    y, sr = librosa.load(chunkfile)
    mfcc = np.array(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13))
    #print(mfcc.shape)
    #mfcc_array.append(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=39))
    if mfcc.shape[0]==13 and mfcc.shape[1]==44:
        a = []
        a.append(mfcc[0])
        a.append(mfcc[12])
        mfcc_array.append(a)
    #mfcc_array.append(np.pad(mfcc, (0,-mfcc.shape[0]), 'constant', constant_values=(0)))
    
mfcc_array = np.array(mfcc_array)

print(mfcc_array.shape)

#mfcc_array = tf.keras.utils.normalize(mfcc_array, order=2)
#mfcc_array = StandardScaler().fit_transform(mfcc_array)

z = mfcc_array.reshape(mfcc_array.shape[0], 2, 44, 1)

s=0
i=0
t=0

for f in z:
    #pc = model.predict_classes(f.reshape(1,39,44,1), batch_size=1)
    pp = model.predict(f.reshape(1,2,44,1), batch_size=1)
    pp = np.around(pp,decimals=2)
    #print("chunk",str(i),"-->",pc,"-->",pp)
    if pp[0][0] == 1.0:
        s+=1
    t+=pp[0][0]
    i+=1
    print("chunk",str(i),"-->",pp)

print(s/len(z)*100)
print(s)
print(len(z))
print(t/len(z)*100)


ff
[]
(133, 2, 44)
chunk 1 --> [[ 0.99000001]]
chunk 2 --> [[ 0.99000001]]
chunk 3 --> [[ 0.97000003]]
chunk 4 --> [[ 0.82999998]]
chunk 5 --> [[ 0.99000001]]
chunk 6 --> [[ 0.98000002]]
chunk 7 --> [[ 0.56]]
chunk 8 --> [[ 0.99000001]]
chunk 9 --> [[ 0.99000001]]
chunk 10 --> [[ 0.99000001]]
chunk 11 --> [[ 0.99000001]]
chunk 12 --> [[ 0.98000002]]
chunk 13 --> [[ 0.99000001]]
chunk 14 --> [[ 0.99000001]]
chunk 15 --> [[ 0.97000003]]
chunk 16 --> [[ 0.97000003]]
chunk 17 --> [[ 0.99000001]]
chunk 18 --> [[ 0.99000001]]
chunk 19 --> [[ 0.99000001]]
chunk 20 --> [[ 0.99000001]]
chunk 21 --> [[ 0.99000001]]
chunk 22 --> [[ 0.97000003]]
chunk 23 --> [[ 0.99000001]]
chunk 24 --> [[ 0.99000001]]
chunk 25 --> [[ 0.98000002]]
chunk 26 --> [[ 0.99000001]]
chunk 27 --> [[ 0.99000001]]
chunk 28 --> [[ 0.99000001]]
chunk 29 --> [[ 0.99000001]]
chunk 30 --> [[ 0.98000002]]
chunk 31 --> [[ 0.99000001]]
chunk 32 --> [[ 0.99000001]]
chunk 33 --> [[ 0.99000001]]
chunk 34 --> [[ 0.99000001]]
chunk 35 -

In [1]:
from keras.models import load_model
model1 = load_model('/home/mansi/anaconda3/beproject/stutter_det/models/best_model_pro.h5')
model1.summary()

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 2, 20, 32)         192       
_________________________________________________________________
activation_13 (Activation)   (None, 2, 20, 32)         0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 2, 8, 32)          5152      
_________________________________________________________________
activation_14 (Activation)   (None, 2, 8, 32)          0         
_________________________________________________________________
reshape_5 (Reshape)          (None, 16, 32)            0         
_________________________________________________________________
gru1 (GRU)                   (None, 16, 32)            6240      
_________________________________________________________________
gru2 (GRU)                   (None, 32)                6240      
__________

In [2]:
import librosa
#import keras
import numpy as np
import tensorflow as tf
from pydub import AudioSegment
#from pydub.silence import split_on_silence
#import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
np.set_printoptions(threshold=np.nan)

arr = ['M_0760_15y3m_1',
       'M_1202_10y7m_1',
       'M_1215_11y0m_1',
       'M_1202_10y11m_1',
       'M_1202_10y7m_1',
       'M_0760_12y3m_1',
       'M_0077_11y2m_1',
       'M_0061_18y5m_2',
       'M_0061_18y5m_1',
       'F_1211_11y3m_1',
       'F_0101_14y8m_1',
       'M_1202_10y11m_1',
       'M_1216_11y1m_1',
       'M_0048_11y1m_2',
       'M_0048_11y5m_1',
       'M_0048_11y6m_1',
       'M_0053_10y8m_1',
       'M_0064_11y11m_1',
       'M_0064_11y5m_1'
      ]

arr2 = ["M_1206_12y3m_1",
"M_1202_10y7m_1",
"M_1023_11y6m_1",
"M_1022_10y9m_2",
"M_1213_11y3m_1",
"M_0253_14y6m_1",
"M_0104_15y4m_1",
"M_1022_10y10m_1",
"M_0880_15y2m_1",
"M_0760_12y3m_1",
"M_1215_11y0m_1",
"M_1214_11y8m_1",
"M_1207_13y5m_1",
"M_0760_15y3m_1",
"M_0760_12y3m_1",
"M_1023_10y10m_1",
"M_0545_15y3m_1",
"M_0541_12y3m_1",
"M_0104_18y1m_1",
"M_1216_11y1m_1",
"M_1205_13y1m_1",
"M_1040_12y6m_1",
"M_0880_14y6m_2"]

files = os.listdir("recordings")

for audio in arr2 :
    #if audio.endswith('.wav'):
    print(audio)
    #print(str(i)+'.wav')
    #sound_file = AudioSegment.from_wav("/home/mansi/anaconda3/beproject/stutter_det/recordings/"+audio)
    sound_file = AudioSegment.from_wav("/home/mansi/anaconda3/beproject/stutter_det/test_rep/"+audio+'.wav')
    audio_chunks = sound_file[::1000]

    mfcc_array = []

    for i, chunk in enumerate(audio_chunks):
        chunkfile = "chunks_test/chunk{0}.wav".format(i)
        chunk.export(chunkfile, format="wav")
        y, sr = librosa.load(chunkfile)
        mfcc = np.array(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13))
        if mfcc.shape[0]==13 and mfcc.shape[1]==44:
            a = []
            a.append(mfcc[0])
            a.append(mfcc[12])
            mfcc_array.append(a)

    mfcc_array = np.array(mfcc_array)
    #print(mfcc_array.shape)
    
    z = mfcc_array.reshape(mfcc_array.shape[0], 2, 44, 1)
    s=0
    i=0
    t=0
    for f in z:
        pp = model1.predict(f.reshape(1,2,44,1), batch_size=1)
        pp = np.around(pp,decimals=2)
        
        s+=pp[0][0]
        if pp[0][0] > 0.5:
            t+=pp[0][0]
        #print("chunk",str(i),"-->",pp)
        i+=1
    print(t/len(z)*100)
    print(s/len(z)*100)

M_1206_12y3m_1
18.6214954218
19.5046730009
M_1202_10y7m_1
18.4098360968
20.9016393687
M_1023_11y6m_1
8.55371897871
8.84297517367
M_1022_10y9m_2
26.1653543457
27.3779527541
M_1213_11y3m_1
39.5792350417
41.8743169984
M_0253_14y6m_1
44.3285025548
45.1980677744
M_0104_15y4m_1
51.7256100003
52.6402441489
M_1022_10y10m_1
19.6504065176
20.4552845742
M_0880_15y2m_1
28.1390728935
30.013245095
M_0760_12y3m_1
61.7670457133
64.9886366594
M_1215_11y0m_1
35.7157895753
37.6157895672
M_1214_11y8m_1
27.6562500373
28.9375000133
M_1207_13y5m_1
12.7213114923
13.7978142188
M_0760_15y3m_1
71.1739132
72.5341616426
M_0760_12y3m_1
61.7670457133
64.9886366594
M_1023_10y10m_1
18.1755726119
20.0534351853
M_0545_15y3m_1
46.18300667
47.8823530762
M_0541_12y3m_1
27.1804511995
28.7518797862
M_0104_18y1m_1
14.0800000032
16.5866666784
M_1216_11y1m_1
31.9658120473
35.2991453921
M_1205_13y1m_1
22.1707317403
23.5284553311
M_1040_12y6m_1
22.9268293071
24.2439024696
M_0880_14y6m_2
13.8846154396
14.38461545
